In [1]:
import time
import socket
from queue import Queue 
from threading import Thread, Condition
from ipywidgets import widgets, Layout, Output
from IPython.display import display
import random

In [2]:
class EXP_Output:
    def __init__(self):
        #self.out = widgets.Output(layout={'border': '1px solid black'})
        #display(self.out)
        #with self.out:
        #print("Experiment outputs")

        self.Queue_Print = Queue()
        self.thread = Thread(target=self.loop_Print)
        self.thread.start()

    def print(self, msg):
        text_out = time.strftime("%H:%M:%S", time.localtime()) + "> "
        self.Queue_Print.put(text_out+msg)

    def loop_Print(self):
        while True:
            msg = self.Queue_Print.get()
            #with self.out:
            print(msg)
            self.Queue_Print.task_done()

In [3]:
class ALE_TextInput:
    
    def __init__(self, exp_out):
        
        self.Queue_User = Queue()
        self.exp_out = exp_out

        # Create a text input widget
        self.Text = widgets.Text(
                        value='',
                        placeholder='Type something and press enter',
                        description='To send:',
                        disabled=False
                    )

        # create a button object
        self.Botton =  widgets.Button(
                            description='Submit',
                            disabled=False,
                            button_style='', 
                            tooltip='Click me to submit a string',
                        )

        # Define a function to handle the click action on the button
        def on_submit_button_clicked(b):

            global thread_running
            
            msg = self.Text.value
            #self.exp_out.print("ALE_0: received input: "+str(msg))
                
            if thread_running == True:
                if msg == "end":
                    thread_running = False
                
                for i in range(20):
                    self.Queue_User.put(msg+str(i))

            self.Text.value = ''  # Clear the input field after submission

        # Attach the event to the button
        self.Botton.on_click(on_submit_button_clicked)
        
        # Display the widgets
        display(self.Text, self.Botton)
        
    # to get a message from user queue. this function can block the thread
    def get(self):
        return self.Queue_User.get()

In [4]:
class ALE_TR:
    
    def __init__(self, name, upper_Tx, lower_TR, exp_out):
        
        self.Name     = name
        self.Upper_Tx = upper_Tx  # must provide get()
        self.Lower_TR = lower_TR  # must provide send(msg), and receive()
        self.exp_out  = exp_out
        
    def loop_Tx(self):

        global thread_running
        c = 0
        self.exp_out.print(self.Name + ": loop_Tx starting")
    
        while (thread_running == True):

            c = c + 1

            # get text from Upper_Tx, which must provide a get method
            # this thread is blocked here
            msg = self.Upper_Tx.get()
            
            text_out = self.Name + " Tx: message " + str(c) + ": " + str(msg)
            self.exp_out.print(text_out)

            # add the text to the queue
            self.Lower_TR.send(msg)
            
    def loop_Rx(self):
        
        global thread_running
        c = 0
        self.exp_out.print(self.Name + ": loop_Rx starting")
    
        while (thread_running == True):

            c = c + 1

            # get message from a lower layer
            # this thread is blocked here
            msg = self.Lower_TR.receive()
            
            text_out = self.Name + " Rx: message " + str(c) + ": " + str(msg)
            self.exp_out.print(text_out)

In [5]:
# state of DLE
STATE_READY_TO_SEND = 0 # ready to send a packet to the DLE entity in another node through lower layer
STATE_WAITING_ACK   = 1 # waiting for ACK from the DLE entity in another node

# events of DLE
EVENT_UPPER_TX  = 0 # upper layer wants to send a packet
EVENT_LOWER_DAT = 1 # lower layer forwards an incoming data packet
EVENT_LOWER_ACK = 2 # lower layer forwards an incoming data packet
EVENT_TIMEOUT   = 3 # timer timeout
        
class DLE_TR_FSM:
    
    def __init__(self, name, lower_TR, exp_out):
        
        self.Name      = name
        self.Lower_TR  = lower_TR   # must provide send(msg) and receive()
        self.Queue_Tx  = Queue()
        self.Queue_Rx  = Queue()
        self.exp_out   = exp_out
        
        # create a queue for Finite State Machine
        self.Queue_FSM = Queue()
        
        # create a flag = condition to start sending
        self.cv_TxOp = Condition()
        self.TxOp    = True

        # init state to STATE_WAITING_UPPER
        self.State = STATE_READY_TO_SEND
        
        self.Procedure = [[self.FSM_upper_Tx, self.FSM_abnormal],
                          [self.FSM_lower_Rx, self.FSM_lower_Rx],
                          [self.FSM_abnormal, self.FSM_lower_Rx_ack],
                          [self.FSM_abnormal, self.FSM_timeout]] 
        

        # new features for retransmission
        # timer
        self.cv_Timer      = Condition()
        self.timer_active  = False
        self.timer_counter = 0
        
        # transmission 
        self.tx_buffer  = ''
        self.tx_seq_num = '0'
        
        # reception
        self.tx_ack_num = '0'
        
        
    def loop_Tx(self):

        global thread_running
        c = 0
        self.exp_out.print(self.Name + ": loop_Tx starting")
    
        while (thread_running == True):

            # waiting for the ready to sent signal
            with self.cv_TxOp: 
                while (self.TxOp != True): 
                    self.cv_TxOp.wait()
            
                self.TxOp = False
                
                c = c + 1

                # get a message from queue
                msg = self.Queue_Tx.get()
                text_out = self.Name + " Tx: message " + str(c) + ": " + str(msg)
                self.exp_out.print(text_out)
            
                self.event_add(EVENT_UPPER_TX, msg) 

    def loop_Rx(self):
        
        global thread_running
        c = 0
        self.exp_out.print(self.Name + ": loop_Rx starting")
    
        while (thread_running == True):

            c = c + 1

            # get message from a lower layer
            # this thread is blocked here
            msg = self.Lower_TR.receive()
            
            text_out = self.Name + " Rx: message " + str(c) + ": " + str(msg)
            self.exp_out.print(text_out)
            
            # add an event for FSM, assuming that msg is a sequence of bytes
            msg_type = msg[0]
            if (msg_type == '0'):
                self.event_add(EVENT_LOWER_DAT, msg[1:])
            elif (msg_type == '1'):
                self.event_add(EVENT_LOWER_ACK, msg[1:])
            else:
                text_out = self.Name + " Rx: message type unknown " + msg_type
                self.exp_out.print(text_out)

    def receive(self):
        return self.Queue_Rx.get()
    
    def send(self, msg):
        self.Queue_Tx.put(msg)
        
    def event_add(self, ev_type, msg):
        
        # preparing an event
        if (isinstance(msg, str)):
            msg = msg.encode()
        
        # the event is a sequence of bytes
        event = ev_type.to_bytes(1, "big")+msg
        
        # add event to queue
        self.Queue_FSM.put(event)        
        

    def loop_FSM(self):
        
        global thread_running
        self.exp_out.print(self.Name + ": loop_FSM starting")
        
        while (thread_running == True):
            
            # get the next event
            event = self.Queue_FSM.get()
            
            ev_type = event[0]
            
            text_out = self.Name + " FSM: state: " + str(self.State)+" event type: " + str(ev_type)
            self.exp_out.print(text_out)
            
            # process the event
            msg = event[1:].decode('utf-8') # message becomes a string
            self.Procedure[ev_type][self.State](msg)
        
    def FSM_abnormal(self, msg):
        
        text_out = self.Name + " FSM: error! " + msg
        self.exp_out.print(text_out)
    
    def FSM_upper_Tx(self, msg):   
        
        text_out = self.Name + " FSM: to send: " + msg
        self.exp_out.print(text_out)

        # buffer the message (for retransmission)
        self.tx_buffer = msg
        
        # prepare to send a data message
        msg = '0' + self.tx_seq_num + msg     # inicating a new packet and its sequence number
        self.Lower_TR.send(msg)

        # waiting for ACK
        self.State = STATE_WAITING_ACK
        
        # start timer
        with self.cv_Timer:
            self.timer_active  = True
            self.timer_counter = 10
            self.cv_Timer.notify()    
    
    def FSM_lower_Rx(self, msg):

        # get the sequence number
        seq = msg[0]
        msg = msg[1:]
        
        # received a new data packet        
        text_out = self.Name + " FSM: received: " + msg
        self.exp_out.print(text_out)

        if (seq == self.tx_ack_num):
            
            # put the message in a receiving queue
            self.Queue_Rx.put(msg)
            
            self.tx_ack_num = chr(ord(self.tx_ack_num) + 1)
            if (self.tx_ack_num == chr(ord('9') + 1)):
                self.tx_ack_num = '0'
        
        else:
            text_out  = self.Name + " FSM: received received frame " + seq
            text_out += " but expected " + self.tx_ack_num
            self.exp_out.print(text_out)
            

        # to send an ACK
        self.Lower_TR.send('1'+seq)

                
    def FSM_lower_Rx_ack(self, msg):
        
        ack = msg[0] # get the ack number
        if (ack != self.tx_seq_num):
            
            # the received ack does not match the seq
            text_out = "ACK # " + ack + " does not match local seq # " + self.tx_seq_num
            self.exp_out.print(text_out)
            return

        # stop timer
        self.timer_active = False
        
        # inc the seq
        self.tx_seq_num = chr(ord(self.tx_seq_num) + 1)
        if (self.tx_seq_num == chr(ord('9') + 1)):
                self.tx_seq_num = '0'
        
        # ready to send the next upper layer packet
        with self.cv_TxOp:
            self.State = STATE_READY_TO_SEND
            self.TxOp  = True
            self.cv_TxOp.notify()
            
    def FSM_timeout(self, msg):
        
        text_out = self.Name + " FSM: to resend frame " + self.tx_seq_num + " " + self.tx_buffer
        self.exp_out.print(text_out)
        
        # prepare to send a data message
        msg = '0' + self.tx_seq_num + self.tx_buffer     # inicating a new packet and its sequence number
        self.Lower_TR.send(msg)
        
        # start timer
        with self.cv_Timer:
            self.timer_active  = True
            self.timer_counter = 10
            self.cv_Timer.notify()

    def loop_timer(self):
        
        global thread_running
        self.exp_out.print(self.Name + ": loop_Timer starting")
        
        while (thread_running == True):
            
            with self.cv_Timer: 
                while (self.timer_active == False): 
                    self.cv_Timer.wait() 
            
                time.sleep(0.5)

                if (self.timer_counter == 0):
                    # add an event for timeout
                    self.event_add(EVENT_TIMEOUT, 'x')
                    self.timer_active = False

                else:
                    self.timer_counter = self.timer_counter - 1

In [6]:
# class PLE_TR:

#     def __init__(self, name, Socket, AP_Tx, AP_Rx, exp_out):
        
#         self.Name      = name
#         self.Socket    = Socket
#         self.AP_Tx     = AP_Tx
#         self.AP_Rx     = AP_Rx
#         self.Queue_Tx  = Queue()
#         self.Queue_Rx  = Queue()
#         self.exp_out   = exp_out
        
#     def loop_Tx(self):
        
#         global thread_running
#         self.exp_out.print(self.Name + ": loop_Tx starting")
    
#         while (thread_running == True):
        
#             # get a message from queue
#             msg = self.Queue_Tx.get()
            
#             text_out = self.Name + " Tx: message: " + str(msg)
#             self.exp_out.print(text_out)
            
#             # sending the message using socket
#             msg_bytes = str.encode(msg)
#             if (random.random() < 0.2):
#                 self.exp_out.print("packet loss")
#             else:
#                 # sending the message using socket
#                 self.Socket.sendto(msg_bytes, self.AP_Tx)

#     def loop_Rx(self):
        
#         global thread_running
#         global bufferSize
#         self.exp_out.print(self.Name + ": loop_Rx starting")
    
#         # binding the socket with the IP and port
#         self.Socket.bind(self.AP_Rx)
        
#         while (thread_running == True):
        
#             # get a message from socket, this thread is blocked here
#             msg_addr = self.Socket.recvfrom(bufferSize)

#             # to emulate a slow receiver
#             time.sleep(0.5)
    
#             msg  = msg_addr[0].decode('utf-8')
#             addr = msg_addr[1]
            
#             text_out = self.Name + " Rx: from " + str(addr) + ": " + str(msg)
#             self.exp_out.print(text_out)
            
#             self.Queue_Rx.put(msg)
            
#     def send(self, msg):
#         self.Queue_Tx.put(msg)
        
#     def receive(self):
#         return self.Queue_Rx.get()

class PLE_TR:

    def __init__(self, name, Socket, AP_Tx, AP_Rx, exp_out):
        
        self.Name      = name
        self.Socket    = Socket
        self.AP_Tx     = AP_Tx
        self.AP_Rx     = AP_Rx
        self.Queue_Tx  = Queue()
        self.Queue_Rx  = Queue()
        self.exp_out   = exp_out
        
    def loop_Tx(self):
        global thread_running
        self.exp_out.print(self.Name + ": loop_Tx starting")

        while (thread_running == True):
            # 1) Get a message from the queue (blocks until available)
            msg = self.Queue_Tx.get()
            self.exp_out.print(self.Name + " Tx: message: " + str(msg))

            # 2) Decide loss & duplication using a single random draw
            #    r in [0,1): 
            #      - r < 0.16   => drop (16%)
            #      - 0.15 <= r < 0.20 => send + duplicate (5%)
            #      - r >= 0.20  => normal single send (80%)
            r = random.random()
            msg_bytes = msg.encode()

            if r < 0.15:
                # Drop
                self.exp_out.print(self.Name + " PLE: packet loss (15%)")
                continue

            # Send once
            self.Socket.sendto(msg_bytes, self.AP_Tx)

            # Optional duplicate
            if r < 0.20:
                self.exp_out.print(self.Name + " PLE: duplicate transmission (5%)")
                self.Socket.sendto(msg_bytes, self.AP_Tx)


    def loop_Rx(self):
        global thread_running, bufferSize
        self.exp_out.print(self.Name + ": loop_Rx starting")

        # Bind once before the receive loop
        self.Socket.bind(self.AP_Rx)

        while (thread_running == True):
            # Block waiting for a frame
            msg_addr = self.Socket.recvfrom(bufferSize)

            # Random delay 0.0 to 3.0 seconds
            delay = random.uniform(0.0, 3.0)
            time.sleep(delay)

            msg  = msg_addr[0].decode('utf-8')
            addr = msg_addr[1]

            # Log the delay for evidence
            self.exp_out.print(self.Name + f" Rx: from {addr}: {msg} (rx_delay={delay:.2f}s)")

            # Hand to upper queue
            self.Queue_Rx.put(msg)

            
    def send(self, msg):
        self.Queue_Tx.put(msg)
        
    def receive(self):
        return self.Queue_Rx.get()

In [ ]:
thread_running = False
bufferSize = 1024

# (0) creating an output object
exp_out = EXP_Output()

# (1) create physical layer entities
AP_local_1  = ("127.0.0.1", 30000)
AP_remote_1 = ("127.0.0.1", 31111)
Socket_1    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_1       = PLE_TR("PLE_Alice", Socket_1, AP_remote_1, AP_local_1, exp_out)

# (2) create date link layer entities
DLE_1 = DLE_TR_FSM("DLE_Alice", PLE_1, exp_out)

# (3) create application layer entities
ALE_0 = ALE_TextInput(exp_out)
ALE_1 = ALE_TR("ALE_Alice", ALE_0, DLE_1, exp_out)

Text(value='', description='To send:', placeholder='Type something and press enter')

Button(description='Submit', style=ButtonStyle(), tooltip='Click me to submit a string')

21:49:30> ALE_Alice: loop_Tx starting
21:49:30> ALE_Alice: loop_Rx starting
21:49:30> DLE_Alice: loop_Tx starting
21:49:30> DLE_Alice: loop_Rx starting
21:49:30> PLE_Alice: loop_Tx starting
21:49:30> PLE_Alice: loop_Rx starting
21:49:30> ALE_Bob: loop_Tx starting
21:49:30> ALE_Bob: loop_Rx starting
21:49:30> DLE_Bob: loop_Tx starting
21:49:30> DLE_Bob: loop_Rx starting
21:49:30> PLE_Bob: loop_Tx starting
21:49:30> PLE_Bob: loop_Rx starting
21:49:30> DLE_Alice: loop_FSM starting
21:49:30> DLE_Bob: loop_FSM starting
21:49:30> DLE_Alice: loop_Timer starting
21:49:30> DLE_Bob: loop_Timer starting


In [8]:
# (4) create physical layer entities
AP_local_2  = ("127.0.0.1", 31111)
AP_remote_2 = ("127.0.0.1", 30000)
Socket_2    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_2       = PLE_TR("PLE_Bob", Socket_2, AP_remote_2, AP_local_2, exp_out)

# (5) create date link layer entities
DLE_2 = DLE_TR_FSM("DLE_Bob", PLE_2, exp_out)

# (6) create application layer entities
ALE_3 = ALE_TextInput(exp_out)
ALE_2 = ALE_TR("ALE_Bob", ALE_3, DLE_2, exp_out)

Text(value='', description='To send:', placeholder='Type something and press enter')

Button(description='Submit', style=ButtonStyle(), tooltip='Click me to submit a string')

In [9]:
# start the loops of all entities
# all loops must be blocked at a certain position

t1_1 = Thread(target = ALE_1.loop_Tx, args = ()) 
t2_1 = Thread(target = ALE_1.loop_Rx, args = ()) 
t3_1 = Thread(target = DLE_1.loop_Tx, args = ())
t4_1 = Thread(target = DLE_1.loop_Rx, args = ())
t5_1 = Thread(target = PLE_1.loop_Tx, args = ()) 
t6_1 = Thread(target = PLE_1.loop_Rx, args = ())
f1_1 = Thread(target = DLE_1.loop_FSM, args = ())
f2_1 = Thread(target = DLE_1.loop_timer, args = ())


t1_2 = Thread(target = ALE_2.loop_Tx, args = ()) 
t2_2 = Thread(target = ALE_2.loop_Rx, args = ()) 
t3_2 = Thread(target = DLE_2.loop_Tx, args = ())
t4_2 = Thread(target = DLE_2.loop_Rx, args = ())
t5_2 = Thread(target = PLE_2.loop_Tx, args = ()) 
t6_2 = Thread(target = PLE_2.loop_Rx, args = ()) 
f1_2 = Thread(target = DLE_2.loop_FSM, args = ())
f2_2 = Thread(target = DLE_2.loop_timer, args = ())

thread_running = True

t1_1.start()
t2_1.start()
t3_1.start()
t4_1.start()
t5_1.start()
t6_1.start()
t1_2.start()
t2_2.start()
t3_2.start()
t4_2.start()
t5_2.start()
t6_2.start()
f1_1.start()
f1_2.start()
f2_1.start()
f2_2.start()

In [10]:
### Measurement Layer Analysis ### ↓↓↓

In [11]:
# Measurement Layer Entity Class
class MLE_TR:
    def __init__(self, name, experiment, lower_TR, exp_out):
        self.Name = name
        self.Experiment = experiment
        self.Lower_TR = lower_TR
        self.exp_out = exp_out

        self.submit = widgets.Button(
                            description=f"Send 100 messages from {self.Name}.",
                            disabled=False,
                            button_style='', 
                            tooltip='Click me to submit a string',
                            layout=Layout(width='500px')
                        )
        self.submit.on_click(self.button_clicked)
        display(self.submit)

    def button_clicked(self, submit):
        # Initialize the CSV file to store experiment data
        # Deletes old data if it already exists
        with open(f"./experiment_outputs/experiment{self.Experiment}.csv", "w") as csv:
            csv.write("experimentId,messageId,currentTime\n")
        
        for i in range(100):
            # CSV Format: experimentId,messageId,currentTime
            msg = f"{self.Experiment},{i}"
            self.Lower_TR.send(msg)
    
    def loop_Rx(self):
        global thread_running
        c = 0
        self.exp_out.print(self.Name + ": MLE loop_Rx starting")
    
        while (thread_running == True):

            c = c + 1

            # get message from a lower layer
            # this thread is blocked here
            msg = self.Lower_TR.receive()

            # Append timestamp of when message was received (in ms)
            msg += f",{time.time_ns()/1_000_000}"
            text_out = self.Name + " MLE Rx: message " + str(c) + ": " + str(msg) 
            self.exp_out.print(text_out)

            # Everytime we successfully receive a message, store it in the CSV
            with open(f"./experiment_outputs/experiment{self.Experiment}.csv", "a") as csv:
                csv.write(str(msg)+"\n")

In [12]:
# Experiment 1: Two Nodes and Two Layers
# MLE_TR
# 
# PLE_TR

thread_running = False
bufferSize = 1024
exp_out = EXP_Output()

# Node 1 
AP_local_3  = ("127.0.0.1", 32222)
AP_remote_3 = ("127.0.0.1", 33000)

Socket_3 = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
PLE_3 = PLE_TR("PLE_Node1", Socket_3, AP_remote_3, AP_local_3, exp_out)
MLE_1 = MLE_TR("MLE_Node1", 1, PLE_3, exp_out)

# Node 2 
AP_local_4  = ("127.0.0.1", 33000)
AP_remote_4 = ("127.0.0.1", 32222)

Socket_4 = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
PLE_4 = PLE_TR("PLE_Node2", Socket_4, AP_remote_4, AP_local_4, exp_out)
MLE_2 = MLE_TR("MLE_Node2", 1, PLE_4, exp_out)


Button(description='Send 100 messages from MLE_Node1.', layout=Layout(width='500px'), style=ButtonStyle(), too…

Button(description='Send 100 messages from MLE_Node2.', layout=Layout(width='500px'), style=ButtonStyle(), too…

21:49:31> PLE_Node1: loop_Tx starting
21:49:31> PLE_Node1: loop_Rx starting
21:49:31> MLE_Node1: MLE loop_Rx starting
21:49:31> PLE_Node2: loop_Tx starting
21:49:31> PLE_Node2: loop_Rx starting
21:49:31> MLE_Node2: MLE loop_Rx starting
21:50:25> PLE_Node1 Tx: message: 1,0
21:50:25> PLE_Node1 Tx: message: 1,1
21:50:25> PLE_Node1 Tx: message: 1,2
21:50:25> PLE_Node1 PLE: packet loss (15%)
21:50:25> PLE_Node1 Tx: message: 1,3
21:50:25> PLE_Node1 Tx: message: 1,4
21:50:25> PLE_Node1 Tx: message: 1,5
21:50:25> PLE_Node1 Tx: message: 1,6
21:50:25> PLE_Node1 Tx: message: 1,7
21:50:25> PLE_Node1 PLE: packet loss (15%)
21:50:25> PLE_Node1 Tx: message: 1,8
21:50:25> PLE_Node1 Tx: message: 1,9
21:50:25> PLE_Node1 Tx: message: 1,10
21:50:25> PLE_Node1 PLE: packet loss (15%)
21:50:25> PLE_Node1 Tx: message: 1,11
21:50:25> PLE_Node1 Tx: message: 1,12
21:50:25> PLE_Node1 Tx: message: 1,13
21:50:25> PLE_Node1 Tx: message: 1,14
21:50:25> PLE_Node1 PLE: duplicate transmission (5%)
21:50:25> PLE_Node1 Tx

In [13]:
# Experiment 2: Two Nodes and Three Layers
# MLE_TR
# 
# DLE_TR
#
# PLE_TR

exp_out = EXP_Output()
# Node 1
AP_local_5 = ("127.0.0.1", 33333)
AP_remote_5 = ("127.0.0.1", 34000)

Socket_5 = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
PLE_5 = PLE_TR("PLE_Node1", Socket_5, AP_remote_5, AP_local_5, exp_out)
DLE_3 = DLE_TR_FSM("DLE_Node1", PLE_5, exp_out)
MLE_3 = MLE_TR("MLE_Node1", 2, DLE_3, exp_out)

# Node 2
AP_local_6 = ("127.0.0.1", 34000)
AP_remote_6 = ("127.0.0.1", 33333)

Socket_6 = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
PLE_6 = PLE_TR("PLE_Node2", Socket_6, AP_remote_6, AP_local_6, exp_out)
DLE_4 = DLE_TR_FSM("DLE_Node2", PLE_6, exp_out)
MLE_4 = MLE_TR("MLE_Node2", 2, DLE_4, exp_out)

Button(description='Send 100 messages from MLE_Node1.', layout=Layout(width='500px'), style=ButtonStyle(), too…

Button(description='Send 100 messages from MLE_Node2.', layout=Layout(width='500px'), style=ButtonStyle(), too…

21:49:31> PLE_Node1: loop_Rx starting
21:49:31> PLE_Node1: loop_Tx starting
21:49:31> DLE_Node1: loop_Rx starting
21:49:31> DLE_Node1: loop_Tx starting
21:49:31> DLE_Node1: loop_FSM starting
21:49:31> DLE_Node1: loop_Timer starting
21:49:31> MLE_Node1: MLE loop_Rx starting
21:49:31> PLE_Node2: loop_Rx starting
21:49:31> PLE_Node2: loop_Tx starting
21:49:31> DLE_Node2: loop_Rx starting
21:49:31> DLE_Node2: loop_Tx starting
21:49:31> DLE_Node2: loop_FSM starting
21:49:31> DLE_Node2: loop_Timer starting
21:49:31> MLE_Node2: MLE loop_Rx starting
21:56:20> DLE_Node1 Tx: message 1: 2,0
21:56:20> DLE_Node1 FSM: state: 0 event type: 0
21:56:20> DLE_Node1 FSM: to send: 2,0
21:56:20> PLE_Node1 Tx: message: 002,0
21:56:20> PLE_Node1 PLE: packet loss (15%)
21:56:26> DLE_Node1 FSM: state: 1 event type: 3
21:56:26> DLE_Node1 FSM: to resend frame 0 2,0
21:56:26> PLE_Node1 Tx: message: 002,0
21:56:26> PLE_Node1 PLE: duplicate transmission (5%)
21:56:26> PLE_Node2 Rx: from ('127.0.0.1', 33333): 002,0 (

In [14]:
# Run the loop functions on threads
x1_1 = Thread(target = PLE_3.loop_Tx, args = ()) 
x1_2 = Thread(target = PLE_3.loop_Rx, args = ()) 
x1_3 = Thread(target = MLE_1.loop_Rx, args=())
x2_1 = Thread(target = PLE_4.loop_Tx, args = ()) 
x2_2 = Thread(target = PLE_4.loop_Rx, args = ()) 
x2_3 = Thread(target = MLE_2.loop_Rx, args=())

x3_1 = Thread(target = PLE_5.loop_Rx, args = ())
x3_2 = Thread(target = PLE_5.loop_Tx, args = ())
x3_3 = Thread(target = DLE_3.loop_Rx, args = ())
x3_4 = Thread(target = DLE_3.loop_Tx, args = ())
x3_5 = Thread(target = DLE_3.loop_FSM, args = ())
x3_6 = Thread(target = DLE_3.loop_timer, args = ())
x3_7 = Thread(target = MLE_3.loop_Rx, args = ())
x4_1 = Thread(target = PLE_6.loop_Rx, args = ())
x4_2 = Thread(target = PLE_6.loop_Tx, args = ())
x4_3 = Thread(target = DLE_4.loop_Rx, args = ())
x4_4 = Thread(target = DLE_4.loop_Tx, args = ())
x4_5 = Thread(target = DLE_4.loop_FSM, args = ())
x4_6 = Thread(target = DLE_4.loop_timer, args = ())
x4_7 = Thread(target = MLE_4.loop_Rx, args = ())

thread_running = True

x1_1.start()
x1_2.start()
x1_3.start()
x2_1.start()
x2_2.start()
x2_3.start()
x3_1.start()
x3_2.start()
x3_3.start()
x3_4.start()
x3_5.start()
x3_6.start()
x3_7.start()
x4_1.start()
x4_2.start()
x4_3.start()
x4_4.start()
x4_5.start()
x4_6.start()
x4_7.start()

In [15]:
### Experiment Insights Calculators ### ↓↓↓
# Note: This requires each experiment to be ran atleast once.

In [16]:
out = Output()

def exp_1_insights(exp1):
    with out:
        with open("./experiment_outputs/experiment1.csv", "r") as csv:
            message_count = 0
            messages_duped = 0
            message_ids = []
            message_times = []
            for line in csv:
                expId,msgId,currentTime = line.split(",")
                if expId == "experimentId":
                    continue
                else:
                    message_count += 1
                    message_ids.append(int(msgId))
                    message_times.append(float(currentTime))
            
            message_ids_set = set(message_ids)
            messages_duped = len(message_ids) - len(message_ids_set)

            times_substracted = 0
            for i in range(0,message_count-2):
                times_substracted += message_times[i+1] - message_times[i]
            
            avg_time_delay = times_substracted / message_count

            print("*Experiment 1*\n")
            print(f"Messages Received: {message_count} out of 100 \nMessages Lost: {100-message_count}\nMessages Duped: {messages_duped}\nAverage Time Delay: {avg_time_delay} ms\n")

def exp_2_insights(exp2):
    with out:
        with open("./experiment_outputs/experiment2.csv", "r") as csv:
            message_count = 0
            messages_duped = 0
            message_ids = []
            message_times = []
            for line in csv:
                expId,msgId,currentTime = line.split(",")
                if expId == "experimentId":
                    continue
                else:
                    message_count += 1
                    message_ids.append(int(msgId))
                    message_times.append(float(currentTime))
            
            message_ids_set = set(message_ids)
            messages_duped = len(message_ids) - len(message_ids_set)

            times_substracted = 0
            for i in range(0,message_count-2):
                times_substracted += message_times[i+1] - message_times[i]
            avg_time_delay = times_substracted / message_count

            print("*Experiment 2*\n")
            print(f"Messages Received: {message_count} out of 100 \nMessages Lost: {100-message_count}\nMessages Duped: {messages_duped}\nAverage Time Delay: {avg_time_delay} ms\n")

exp1 = widgets.Button(description="Insights from experiment 1.",
                    disabled=False,
                    button_style='', 
                    tooltip='Click me for experiment 1 insights.',
                    layout=Layout(width='500px'))
exp1.on_click(exp_1_insights)
display(exp1)
exp2 = widgets.Button(description="Insights from experiment 2.",
                    disabled=False,
                    button_style='', 
                    tooltip='Click me for experiment 2 insights.',
                    layout=Layout(width='500px'))
exp2.on_click(exp_2_insights)
display(exp2)

display(out)


Button(description='Insights from experiment 1.', layout=Layout(width='500px'), style=ButtonStyle(), tooltip='…

Button(description='Insights from experiment 2.', layout=Layout(width='500px'), style=ButtonStyle(), tooltip='…

Output()